In [ ]:
#!/usr/bin/env python
# coding: utf-8

import psycopg2
from groq import Groq
import json
import sys
import os

try:
    GROQ_API_KEY = os.getenv("GROQ_API_KEY", "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")
    if "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" in GROQ_API_KEY:
        print("AVISO: Usando chave de API de placeholder. Por favor, configure sua chave da Groq.")
    
    groq_client = Groq(api_key=GROQ_API_KEY)

except Exception as e:
    print(f"Erro ao configurar o cliente da Groq. Verifique sua chave. Erro: {e}")
    sys.exit(1)

DB_CONFIG = {
    "host": "172.22.0.40",
    "port": 5432,
    "database": "metrics",
    "user": "admin",
    "password": "admin123"
}

def get_db_connection():
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        return connection
    except psycopg2.OperationalError as e:
        print(f"Erro Crítico: Não foi possível conectar ao banco de dados em {DB_CONFIG['host']}.")
        print(f"Detalhes: {e}")
        return None

def get_available_tables(connection) -> list:
    tables = []
    with connection.cursor() as cur:
        cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_type = 'BASE TABLE'
              AND table_schema NOT IN ('pg_catalog', 'information_schema')
        """)
        for table in cur.fetchall():
            tables.append(table[0])
    return tables
from psycopg2 import sql

def get_metrics_from_db(connection, table_name):
    metrics = []
    with connection.cursor() as cur:
        try:
            query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name))
            cur.execute(query)
            column_names = [desc[0] for desc in cur.description]
            for row in cur.fetchall():
                metrics.append(dict(zip(column_names, row)))
        except Exception as e:
            print(f"Erro ao buscar dados da tabela: {e}")
            return None
    return metrics

def get_llama_analysis(table_name: str, metrics_data: list[dict], user_question: str) -> str:
    json_data_string = json.dumps(metrics_data, indent=2, default=str)
    print(json_data_string)
    system_prompt = """
    **Persona:** Você é um engenheiro SRE (Site Reliability Engineer) sênior, especialista em análise de performance de redes e monitoramento com Prometheus.
    Sua tarefa é analisar os dados fornecidos e responder à pergunta do usuário.
    """
    
    user_prompt = f"""
    **Contexto:** Eu extraí os seguintes dados da tabela de métricas `{table_name}` de um banco de dados. Os dados estão em formato de uma lista de dicionários Python (JSON), onde cada dicionário representa uma linha da tabela.
    
    Dados da Tabela `{table_name}`:
    ```json
    {json_data_string}
    ```
    
    **Tarefa:** Com base EXCLUSIVAMENTE nos dados fornecidos, responda à seguinte pergunta do usuário de forma clara, objetiva e em português. Comece sempre pelos pontos mais críticos e relevantes.

    **Pergunta do Usuário:** "{user_question}"
    
    **Instruções Adicionais:**
    - Use Markdown para formatar a resposta para melhor clareza.
    - Se a pergunta for ampla como "analise a performance" ou "faça um resumo", identifique as métricas mais importantes, apresente seus valores e explique o que significam.
    - Se encontrar valores que indiquem falhas, erros ou anomalias, destaque-os claramente.
    """
    print(system_prompt)
    try:
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": user_prompt,
                }
            ],
            model="llama3-70b-8192", 
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Ocorreu um erro ao contatar a API da Groq: {e}"

def main():
    conn = get_db_connection()
    if not conn:
        sys.exit(1)

    try:
        tables = get_available_tables(conn)
        if not tables:
            print("Nenhuma tabela de métricas encontrada no banco de dados.")
            return

        print("Tabelas de métricas disponíveis:")
        for t in tables:
            print(f"- {t}")
        
        selected_table = input("\nQual tabela você gostaria de consultar? ")
        if selected_table not in tables:
            print(f"Erro: A tabela '{selected_table}' não existe. Encerrando.")
            return

        print(f"\nBuscando dados da tabela '{selected_table}'...")
        metrics = get_metrics_from_db(conn, selected_table)
        
        if metrics is None:
            return

        print(f"{len(metrics)} registros encontrados.")
        print("\n" + "="*25 + "\n")

        question = input(f"Faça sua pergunta sobre os dados da tabela '{selected_table}': ")

        print("\n--- Análise do Chatbot Llama 3 (via Groq) ---")
        analysis = get_llama_analysis(selected_table, metrics, question) 
        print(analysis)

    finally:
        if conn:
            conn.close()
            print("\nConexão com o banco de dados fechada.")

if __name__ == "__main__":
    main()

AVISO: Usando chave de API de placeholder. Por favor, configure sua chave da Groq.
Tabelas de métricas disponíveis:
- metric
- cache_inval_hypertable
- cache_inval_bgw_job
- cache_inval_extension
- hypertable
- tablespace
- dimension
- dimension_slice
- chunk
- chunk_constraint
- chunk_index
- chunk_column_stats
- telemetry_event
- continuous_agg
- continuous_aggs_bucket_function
- continuous_aggs_invalidation_threshold
- continuous_aggs_watermark
- continuous_aggs_hypertable_invalidation_log
- continuous_aggs_materialization_invalidation_log
- compression_algorithm
- compression_settings
- compression_chunk_size
- continuous_agg_migrate_plan
- continuous_agg_migrate_plan_step
- metadata
- bgw_job
- bgw_job_stat
- bgw_job_stat_history
- bgw_policy_chunk_stats
- _hyper_1_1_chunk

Buscando dados da tabela 'metric'...
136 registros encontrados.



--- Análise do Chatbot Llama 3 (via Groq) ---
[
  {
    "time": "2025-07-10 22:51:45+00:00",
    "fivegs_amffunction_rm_registeredsubnbr": 0,
 